## Schritt 0: Benötigte Tools installieren

In [1]:
!pip install tqdm requests gensim

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Schritt 1: Word2vec-Index herunterladen

https://devmount.github.io/GermanWordEmbeddings/

In [2]:
from tqdm import tqdm_notebook as tqdm 
import requests
import math
import os
import zipfile

if not os.path.exists("german.model"):

    url = "https://downloads.codingcoursestv.eu/037%20-%20neuronale%20netze/german.model.zip"
    # Streaming, so we can iterate over the response.
    r = requests.get(url, stream=True)

    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0));
    
    print("Downloading " + str(total_size / 1024 / 1024) + " MB.")
    
    block_size = 1024
    with open('german.model.zip', 'wb') as f:
        for data in tqdm(r.iter_content(block_size), total=math.ceil(total_size//block_size), unit='KB', unit_divisor=1024, unit_scale=True):
            f.write(data)
            
    # Extracting .zipFile
    with zipfile.ZipFile("german.model.zip", "r") as zipf:
        zipf.extract("german.model")
        
    # Remove .zip-File (we don't need it anymore)
    os.remove("german.model.zip")
    
    print("Done!")
else:
    print("Datei existiert bereits")

Datei existiert bereits


In [3]:
# word2vec (gensim): https://radimrehurek.com/gensim/models/word2vec.html

In [4]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load_word2vec_format("german.model", binary=True)

In [9]:
# Obama - USA + Russland = Putin

wv.most_similar(positive=["Obama", "Russland"], negative=["USA"])

/Users/jannis/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Putin', 0.7802653908729553),
 ('US-Praesident_Obama', 0.7328896522521973),
 ('Barack_Obama', 0.7064037322998047),
 ('Russlands_Staatschef', 0.6975832581520081),
 ('Wladimir_Putin', 0.6830098628997803),
 ('russischen_Praesidenten', 0.6809293627738953),
 ('US-Praesident_Barack', 0.6808924674987793),
 ('US-Aussenminister_Kerry', 0.6703359484672546),
 ('Kremlchef_Wladimir', 0.6681345105171204),
 ('Russlands_Praesident', 0.6653517484664917)]

In [10]:
wv.most_similar(positive=["Merkel", "USA"], negative=["Deutschland"])

/Users/jannis/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Kanzlerin_Merkel', 0.6972561478614807),
 ('Merkel_CDU', 0.6899685263633728),
 ('Kanzlerin', 0.6769891977310181),
 ('US-Praesident_Obama', 0.6683843731880188),
 ('Obama', 0.650946319103241),
 ('Russlands_Staatschef', 0.6507245302200317),
 ('Bundeskanzlerin', 0.6503758430480957),
 ('Ueberwachung_Handys', 0.6279088258743286),
 ('Angela_Merkel', 0.6278501152992249),
 ('Rande_G20-Gipfels', 0.6272603273391724)]

In [12]:
wv.most_similar(positive=["Mutter"])

/Users/jannis/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Grossmutter', 0.8721050024032593),
 ('Schwester', 0.8409271240234375),
 ('Tochter', 0.8314520716667175),
 ('Vater', 0.8252553343772888),
 ('Tante', 0.8161165714263916),
 ('Oma', 0.8082747459411621),
 ('Frau', 0.7920665144920349),
 ('vierjaehrige_Tochter', 0.7915295362472534),
 ('Stiefmutter', 0.7876834273338318),
 ('Pflegemutter', 0.7844383716583252)]

In [15]:
from keras.models import Sequential

model = Sequential()
model.add(wv.get_keras_embedding())